## Convolutional Neural Network
- MNIST data
- 3 Convolutional layers
- 2 fully conected layers

## 1. Settings

### 1) Import required libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Set hyperparameters

In [3]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
## pil 이미지를 텐서로, 0-9까지의 라벨값이라 바꿔주지 않았다. Download True는 데이터가 없으면 다운로드 하겠다는 뜻
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

Processing...
Done!


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__()) ## train 개수 6만개
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())   ## test개수 6만개

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


In [6]:
img1, label1 = mnist_train.__getitem__(0)
img2, label2 = mnist_test.__getitem__(0)

In [8]:
print(img1.size(), label1) ## 첫번째 데이터는 라벨5 
print(img1.size(), label2) ## 두번째 데이터는 라벨 7

torch.Size([1, 28, 28]) tensor(5)
torch.Size([1, 28, 28]) tensor(7)


### 3) Set DataLoader

In [13]:
## Dataloader를 통해서 batchsize만큼 묶는다.
## shuffle = True , 섞어서 16개씩 묶는다. 
## for문으로 부르면 된다. 

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()   # super는 상속받을때 한번만...?
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),        # batch X 16 X 24 X 24
            nn.ReLU(),
            nn.Conv2d(16,32,5),       # batch X 32 X 20 X 20
            nn.ReLU(),
            nn.MaxPool2d(2,2),        # batch X 32 X 10 X 10
            nn.Conv2d(32,64,5),       # batch X 64 X 6 X 6
            nn.ReLU(),
            nn.MaxPool2d(2,2)         # batch X 64 X 3 X 3
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self,x):               # Forward propagation
        out = self.layer(x)
        out = out.view(batch_size, -1) # FC전에 쫙 펴주는 부분. reshape로 생각하면된다. batchsize 빼고 나머지를 핀다. 
        out = self.fc_layer(out)
        
        return out
    
model = CNN()
## model = CNN().cuda()  

### 2) Loss func & Optimizer

In [15]:
loss_func = nn.CrossEntropyLoss()
optimizer =  torch.optim.SGD(model.parameters(), lr = learning_rate)

## 4. Train

In [16]:
for i in range(num_epoch):
    for j,[image, label] in enumerate(train_loader):
        x = Variable(image)
        y_ = Variable(label)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 ==0:
            print(loss)

tensor(2.2996, grad_fn=<NllLossBackward>)
tensor(2.2901, grad_fn=<NllLossBackward>)
tensor(2.2928, grad_fn=<NllLossBackward>)
tensor(2.2843, grad_fn=<NllLossBackward>)
tensor(2.2936, grad_fn=<NllLossBackward>)
tensor(2.2891, grad_fn=<NllLossBackward>)
tensor(2.2888, grad_fn=<NllLossBackward>)
tensor(2.2873, grad_fn=<NllLossBackward>)
tensor(2.2774, grad_fn=<NllLossBackward>)
tensor(2.2568, grad_fn=<NllLossBackward>)
tensor(2.2413, grad_fn=<NllLossBackward>)
tensor(2.2435, grad_fn=<NllLossBackward>)
tensor(2.2039, grad_fn=<NllLossBackward>)
tensor(2.1574, grad_fn=<NllLossBackward>)
tensor(2.2078, grad_fn=<NllLossBackward>)
tensor(1.9942, grad_fn=<NllLossBackward>)
tensor(1.6938, grad_fn=<NllLossBackward>)
tensor(1.2521, grad_fn=<NllLossBackward>)
tensor(0.9622, grad_fn=<NllLossBackward>)
tensor(1.1286, grad_fn=<NllLossBackward>)
tensor(0.6144, grad_fn=<NllLossBackward>)
tensor(0.7697, grad_fn=<NllLossBackward>)
tensor(0.6317, grad_fn=<NllLossBackward>)
tensor(0.6029, grad_fn=<NllLossBac

## 5. Test

In [19]:
correct = 0
total = 0

for image, label in test_loader:
    x = Variable(image, volatile=True)
    y_ = Variable(label)
    
    output = model.forward(x)
    _, output_index = torch.max(output,1)     
    
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
    
print("Accuracy of Test Data: {}".format(100*correct/total))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data: 91.31999969482422
